In [1]:
import sys
import copy
import pickle
import numpy as np
import pandas as pd

from tqdm import tqdm
from pathlib import Path
from IPython.display import display

sys.path.append(str(Path.cwd().parent))
from src.data import DataBundle, BatteryData
from src.builders import TRAIN_TEST_SPLITTERS
from src.utils.config import import_config

HOME = Path('..').absolute()

In [2]:
folder = HOME / 'workspaces/baselines/sklearn/discharge_model/mix_20/'

In [3]:
%cd $HOME
config_file = next(folder.glob('config*.yaml'))
config = import_config(config_file, ['train_test_split'])['train_test_split']
train_cell_files, test_cell_files = TRAIN_TEST_SPLITTERS.build(config).split()

train_cells = [BatteryData.load(c) for c in tqdm(train_cell_files, desc='Loading train cells')]
test_cells = [BatteryData.load(c) for c in tqdm(test_cell_files, desc='Loading test cells')]

/workspace/battery/nmi_configs


Loading test cells: 100%|██████████| 137/137 [01:16<00:00,  1.80it/s]


Calculate errors for different materials in `MIX-100` dataset.

In [4]:
def read_preds_and_labels(folder):
    # Load predictions
    predictions = []
    for data in (HOME / folder).glob('predictions*'):
        with open(data, 'rb') as f:
            predictions.append(pickle.load(f))

    # Restore the y_true and y_pred
    targets = []
    for pred  in predictions:
        data: DataBundle = pred['data']
        if data.label_transformation is not None:
            y_true = data.label_transformation.inverse_transform(data.test_data.label).cpu().numpy()
            y_pred = data.label_transformation.inverse_transform(pred['prediction'].cpu()).numpy()
        else:
            y_true = data.test_data.label.cpu().numpy()
            y_pred = pred['prediction'].cpu().numpy()
        targets.append((y_true, y_pred))

    return targets

In [5]:
sklearn_baselines = {
    'dummy': 'Training Mean',
    'variance_model': '``Variance\'\' Model',
    'discharge_model': '``Discharge\'\' Model',
    'full_model': '``Full\'\' Model',
    'ridge': 'Ridge Regression',
    'pcr': 'PCR',
    'plsr': 'PLSR',
    'svm': 'SVM',
    'rf': 'Random Forest'
}
nn_baselines = {
    'cnn': 'CNN',
    'mlp': 'MLP',
    'lstm': 'LSTM'
}

dataset = 'mix_100'

data = {}

for method, method_name in tqdm(sklearn_baselines.items(), desc='Sklearn baselines'):
    folder = Path('workspaces/baselines/sklearn') / method / dataset
    data[method_name] = read_preds_and_labels(folder)

for method, method_name in tqdm(nn_baselines.items(), desc='NN baselines'):
    folder = Path('workspaces/baselines/nn_models') / method / dataset
    data[method_name] = read_preds_and_labels(folder)

folder = Path('workspaces/ablation/feature_spaces/all_features') / dataset
data['BatLiNet'] = read_preds_and_labels(folder)

NN baselines: 100%|██████████| 3/3 [00:03<00:00,  1.07s/it]


In [14]:
def get_formatted_rmse(results, indeces):
    scores = []
    for seed_res in results:
        y_true, y_pred = seed_res
        y_true, y_pred = y_true[indeces], y_pred[indeces]

        rmse = np.mean((y_true - y_pred) ** 2) ** 0.5
        scores.append(rmse)

    return f'{np.mean(scores):.0f}±{np.std(scores):.0f}'

In [7]:
from collections import defaultdict

material_map = defaultdict(list)
for indx, cell in enumerate(test_cells):
    material_map[cell.cathode_material].append(indx)

In [16]:
pd.DataFrame({
    material: {
        method: get_formatted_rmse(data[method], test_indeces)
        for method in data
    } for material, test_indeces in material_map.items()
})

,LCO,NMC,LFP
Training Mean,567±0,196±0,626±0
``Variance'' Model,191±0,100±0,598±0
``Discharge'' Model,122±0,137±0,2012±0
``Full'' Model,140±0,70±0,379±0
Ridge Regression,142±0,188±0,446±0
PCR,280±0,164±0,427±0
PLSR,148±0,134±0,422±0
SVM,120±0,97±0,291±0
Random Forest,59±0,73±0,245±0
CNN,127±27,59±7,287±24


In [17]:
materials = set()
for cell in train_cells:
    materials.add(cell.cathode_material)
print(materials)

{'LFP', 'NMC', 'LCO'}


In [8]:
# Function to extract the mean from the string
def extract_mean(value):
    try:
        return float(value.split('±')[0])
    except:
        return np.inf

# Function to highlight the minimum mean value in each row
def highlight_min(s):
    # Extract means for the row
    means = s.apply(extract_mean)
    # Get the index of the minimum mean
    min_index = means.idxmin()
    # Create a series to apply the highlighting
    is_min = pd.Series([''] * len(s), index=s.index)
    is_min[min_index] = 'background-color: black'
    return is_min

for dataset, data_scores in scores.items():
    print(dataset)
    # TODO: fill in NE results
    data_scores = pd.DataFrame(data_scores)
    if dataset == 'MATR-1':
        data_scores.loc[:, ['``Discharge\'\' Model', '``Full\'\' Model']] = '-'
        data_scores.loc[
            ['Root Mean Squared Error', 'Mean Absolute Percentage Error'],
            '``Discharge\'\' Model'
        ] = ['86±0', '10±0']
        data_scores.loc[
            ['Root Mean Squared Error', 'Mean Absolute Percentage Error'],
            '``Full\'\' Model'
        ] = ['100±0', '8±0']
    if dataset == 'MATR-2':
        data_scores.loc[:, ['``Discharge\'\' Model', '``Full\'\' Model']] = '-'
        data_scores.loc[
            ['Root Mean Squared Error', 'Mean Absolute Percentage Error'],
            '``Discharge\'\' Model'
        ] = ['173±0', '9±0']
        data_scores.loc[
            ['Root Mean Squared Error', 'Mean Absolute Percentage Error'],
            '``Full\'\' Model'
        ] = ['214±0', '11±0']
    display(data_scores.style.apply(highlight_min, axis=1))

MATR-1


,Training Mean,``Variance'' Model,``Discharge'' Model,``Full'' Model,Ridge Regression,PCR,PLSR,SVM,Random Forest,CNN,MLP,LSTM,BatLiNet
Mean Absolute Error,239±0,109±0,-,-,83±0,75±0,79±0,107±0,119±0,77±54,101±2,86±5,46±2
Root Mean Squared Error,399±0,136±0,86±0,100±0,117±0,104±0,105±0,140±0,169±0,125±93,162±7,118±7,63±3
Mean Absolute Percentage Error,28±0,15±0,10±0,8±0,11±0,11±0,11±0,15±0,17±0,9±6,12±0,13±1,6±0
Symmetric Mean Absolute Percentage Error,30±0,15±0,-,-,11±0,10±0,11±0,15±0,16±0,10±7,12±0,12±1,6±0
Median Absolute Error,139±0,89±0,-,-,56±0,48±0,62±0,86±0,75±0,48±31,70±4,60±8,33±4
Normalized Mean Absolute Error,13±0,6±0,-,-,4±0,4±0,4±0,6±0,6±0,4±3,5±0,5±0,2±0
Normalized Mean Squared Error,84±0,10±0,-,-,7±0,6±0,6±0,10±0,15±0,13±22,14±1,7±1,2±0


MATR-2


,Training Mean,``Variance'' Model,``Discharge'' Model,``Full'' Model,Ridge Regression,PCR,PLSR,SVM,Random Forest,CNN,MLP,LSTM,BatLiNet
Mean Absolute Error,414±0,136±0,-,-,118±0,164±0,117±0,204±0,164±0,176±95,128±3,157±26,116±8
Root Mean Squared Error,511±0,211±0,173±0,214±0,186±0,243±0,181±0,300±0,240±0,237±108,207±4,233±43,162±10
Mean Absolute Percentage Error,36±0,12±0,9±0,11±0,10±0,14±0,11±0,18±0,14±0,16±8,11±0,14±2,11±1
Symmetric Mean Absolute Percentage Error,46±0,13±0,-,-,10±0,16±0,10±0,19±0,16±0,17±12,11±0,15±2,11±1
Median Absolute Error,342±0,92±0,-,-,86±0,110±0,83±0,129±0,121±0,133±84,72±3,109±15,89±10
Normalized Mean Absolute Error,30±0,10±0,-,-,8±0,12±0,8±0,15±0,12±0,13±7,9±0,11±2,8±1
Normalized Mean Squared Error,187±0,32±0,-,-,25±0,43±0,23±0,65±0,41±0,49±55,31±1,40±14,19±2


HUST


,Training Mean,``Variance'' Model,``Discharge'' Model,``Full'' Model,Ridge Regression,PCR,PLSR,SVM,Random Forest,CNN,MLP,LSTM,BatLiNet
Mean Absolute Error,341±0,319±0,264±0,270±0,649±0,364±0,349±0,291±0,291±0,390±56,336±18,339±21,179±25
Root Mean Squared Error,420±0,398±0,322±0,335±0,1047±0,435±0,431±0,344±0,345±0,482±138,444±3,441±25,268±28
Mean Absolute Percentage Error,18±0,17±0,14±0,14±0,36±0,19±0,18±0,16±0,16±0,22±4,18±1,20±1,10±2
Symmetric Mean Absolute Percentage Error,18±0,17±0,13±0,13±0,37±0,19±0,18±0,16±0,15±0,20±2,18±1,18±1,10±1
Median Absolute Error,332±0,250±0,239±0,226±0,333±0,356±0,307±0,246±0,254±0,323±25,265±48,277±70,125±29
Normalized Mean Absolute Error,25±0,23±0,19±0,20±0,47±0,26±0,25±0,21±0,21±0,28±4,24±1,25±2,13±2
Normalized Mean Squared Error,127±0,115±0,75±0,81±0,794±0,137±0,135±0,86±0,86±0,182±126,143±2,141±16,53±11


MIX-100


,Training Mean,``Variance'' Model,``Discharge'' Model,``Full'' Model,Ridge Regression,PCR,PLSR,SVM,Random Forest,CNN,MLP,LSTM,BatLiNet
Mean Absolute Error,416±0,345±0,378±0,210±0,280±0,264±0,245±0,159±0,136±0,150±10,244±13,159±11,113±18
Root Mean Squared Error,573±0,521±0,1737±0,331±0,395±0,384±0,371±0,257±0,214±0,252±21,461±30,265±15,168±15
Mean Absolute Percentage Error,59±0,39±0,47±0,22±0,30±0,28±0,26±0,18±0,15±0,15±1,28±1,16±1,14±5
Symmetric Mean Absolute Percentage Error,44±0,36±0,26±0,22±0,29±0,27±0,25±0,16±0,14±0,14±1,22±1,15±1,12±3
Median Absolute Error,307±0,223±0,158±0,132±0,207±0,169±0,155±0,100±0,86±0,76±8,113±2,83±9,75±31
Normalized Mean Absolute Error,18±0,15±0,16±0,9±0,12±0,11±0,10±0,7±0,6±0,6±0,10±1,7±0,5±1
Normalized Mean Squared Error,139±0,115±0,1279±0,46±0,66±0,63±0,58±0,28±0,19±0,27±5,90±12,30±3,12±2


MIX-20


,Training Mean,``Variance'' Model,``Discharge'' Model,``Full'' Model,Ridge Regression,PCR,PLSR,SVM,Random Forest,CNN,MLP,LSTM,BatLiNet
Mean Absolute Error,457±0,447±0,81950±0,307±0,546±0,538±0,371±0,318±0,186±0,814±878,326±18,224±29,128±8
Root Mean Squared Error,594±0,600±0,988653±0,437±0,837±0,707±0,482±0,461±0,290±0,6153±10687,519±25,376±61,207±17
Mean Absolute Percentage Error,102±0,96±0,5951±0,54±0,150±0,60±0,75±0,51±0,31±0,77±69,53±2,33±5,18±3
Symmetric Mean Absolute Percentage Error,66±0,65±0,61±0,46±0,70±0,48±0,57±0,46±0,26±0,33±1,44±2,29±3,16±1
Median Absolute Error,388±0,339±0,299±0,190±0,391±0,427±0,272±0,183±0,107±0,125±8,171±19,104±10,68±7
Normalized Mean Absolute Error,22±0,21±0,3914±0,15±0,26±0,15±0,18±0,15±0,9±0,39±42,16±1,11±1,6±0
Normalized Mean Squared Error,168±0,172±0,466778816±0,91±0,334±0,139±0,111±0,101±0,40±0,72619±182957,129±12,69±23,21±3
